In [ ]:
from rmgpy.kinetics.uncertainties import RateUncertainty
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM, Arrhenius
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib


In [ ]:
training_rxn_1 = Reaction(reactants=[Species(label="C10H7", thermo=ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([151.252,192.882,222.756,251.709,287.608,312.21,350.494],'J/(mol*K)'), H298=(635.035,'kJ/mol'), S298=(398.4,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(399.095,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cb-(Cds-Cds)) + group(Cb-H) + group(Cb-H) + group(Cds-CdsCbH) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cds-CdsCtH) + group(Ct-Ct(Cds-Cds)) + group(Ct-CtH) + ring(Benzene) + radical(CbJ)"""), molecule=[Molecule(smiles="C#CC=Cc1[c]cccc1")], molecular_weight=(127.162,'amu'))], products=[Species(label="C10H7-2", thermo=ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([132.013,175.073,209.779,237.275,280.278,310.568,352.738],'J/(mol*K)'), H298=(401.323,'kJ/mol'), S298=(342.062,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(407.409,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cbf-CbCbCbf) + group(Cbf-CbCbCbf) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cb-H) + polycyclic(s2_6_6_naphthalene) + radical(CbJ)"""), molecule=[Molecule(smiles="[c]1ccc2ccccc2c1")], molecular_weight=(127.162,'amu'))], kinetics=Arrhenius(A=(2.992e+11,'s^-1'), n=0.67, Ea=(58.336,'kcal/mol'), T0=(1,'K')), rank=5)
training_rxn_2 =  Reaction(reactants=[Species(label="C10H9-21", thermo=ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([157.569,199.995,235.392,263.634,306.813,337.314,387.229],'J/(mol*K)'), H298=(534.703,'kJ/mol'), S298=(427.312,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(444.824,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CbCsHH) + group(Cs-CtCsHH) + group(Cb-Cs) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Cb-H) + group(Ct-CtCs) + group(Ct-CtH) + ring(Benzene) + radical(CbJ)"""), molecule=[Molecule(smiles="C#CCCc1[c]cccc1")], molecular_weight=(129.178,'amu'))], products=[Species(label="C10H9-22", thermo=ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([143.803,187.641,225.453,257.683,308.277,344.13,392.633],'J/(mol*K)'), H298=(359.57,'kJ/mol'), S298=(361.477,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(457.296,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CbCsHH) + group(Cs-(Cds-Cds)CsHH) + group(Cb-Cs) + group(Cb-(Cds-Cds)) + group(Cds-CdsCsH) + group(Cb-H) + group(Cds-CdsCbH) + group(Cb-H) + group(Cb-H) + group(Cb-H) + polycyclic(s2_6_6_ben_ene_1) + radical(Cds_S)"""), molecule=[Molecule(smiles="[C]1=Cc2ccccc2CC1")], molecular_weight=(129.178,'amu'))], kinetics=Arrhenius(A=(6.09e+08,'s^-1'), n=0.695, Ea=(6.499,'kcal/mol'), T0=(1,'K')), rank=5) 


In [ ]:
BM_fit_2 = Reaction(kinetics = ArrheniusBM(A=(2.992e+11,'s^-1'), n=0.67, w0=(289.5,'kJ/mol'), E0=(289.5,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]"""))
BM_fit_1 = Reaction(kinetics = ArrheniusBM(A=(6.09e+08,'s^-1'), n=0.695, w0=(289.5,'kJ/mol'), E0=(90.5131,'kJ/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Fitted to 1 reactions at temperatures: [ 300.  500.  600.  700.  800.  900. 1000. 1100. 1200. 1500. 2000.]"""))


Tref=1000

print(BM_fit_2.kinetics.to_arrhenius(training_rxn_2.get_enthalpy_of_reaction(Tref)).get_rate_coefficient(Tref))  #problem, always becomes nan
print(BM_fit_1.kinetics.to_arrhenius(training_rxn_1.get_enthalpy_of_reaction(Tref)).get_rate_coefficient(Tref))

In [ ]:
BM_fit_2.kinetics.to_arrhenius(training_rxn_2.get_enthalpy_of_reaction(Tref)).get_rate_coefficient(1000)